In [1]:
from azureml.core import ComputeTarget, Dataset, Datastore, Experiment, Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.core.graph import PipelineParameter
from azureml.pipeline.steps import AutoMLStep, PythonScriptStep

from azureml.train.automl import AutoMLConfig

import os

/usr/local/bin/miniconda3/envs/azureml/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
ws = Workspace.from_config(auth=InteractiveLoginAuthentication(tenant_id=os.environ["AML_TENANT_ID"]))
ws

Workspace.create(name='pipelines', subscription_id='65a1016d-0f67-45d2-b838-b8f373d6d52e', resource_group='laobri-ml')

In [3]:
compute_name = "cpu-compute3"
if not compute_name in ws.compute_targets :
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                min_nodes=0,
                                                                max_nodes=1)
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # Show the result
    print(compute_target.get_status().serialize())

In [4]:
compute = AmlCompute(ws, compute_name)
print(compute)

AmlCompute(workspace=Workspace.create(name='pipelines', subscription_id='65a1016d-0f67-45d2-b838-b8f373d6d52e', resource_group='laobri-ml'), name=cpu-compute3, id=/subscriptions/65a1016d-0f67-45d2-b838-b8f373d6d52e/resourceGroups/laobri-ml/providers/Microsoft.MachineLearningServices/workspaces/pipelines/computes/cpu-compute3, type=AmlCompute, provisioning_state=Succeeded, location=westus, tags=None)


In [5]:
datastore = ws.get_default_datastore()

In [6]:
aml_run_config = RunConfiguration()
aml_run_config.target = compute

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
aml_run_config.environment.python.user_managed_dependencies = False

# Specify CondaDependencies obj, add necessary packages
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn', 'pyarrow'], 
    pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]'], 
    pin_sdk_version=False)

## Step 0: Grab an open dataset and register it

This is baseline data. If the `Dataset` does not exist, create and register it. Not a part of the Pipeline.

In [7]:
if not 'titanic_ds' in ws.datasets.keys() :
    # create a TabularDataset from Titanic training data
    web_paths = ['https://dprepdata.blob.core.windows.net/demo/Titanic.csv',
                 'https://dprepdata.blob.core.windows.net/demo/Titanic2.csv']
    titanic_ds = Dataset.Tabular.from_delimited_files(path=web_paths)

    titanic_ds.register(workspace = ws,
                                     name = 'titanic_ds',
                                     description = 'new titanic training data',
                                     create_new_version = True)

titanic_ds = Dataset.get_by_name(ws, 'titanic_ds')

In [8]:
type(titanic_ds)

azureml.data.tabular_dataset.TabularDataset

In [9]:
if not 'titanic_files_ds' in ws.datasets.keys() :
    # create a TabularDataset from Titanic training data
    web_paths = ['https://dprepdata.blob.core.windows.net/demo/Titanic.csv',
                 'https://dprepdata.blob.core.windows.net/demo/Titanic2.csv']
    titanic_ds = Dataset.File.from_files(path=web_paths)

    titanic_ds.register(workspace = ws,
                                     name = 'titanic_files_ds',
                                     description = 'File Dataset of titanic training data',
                                     create_new_version = True)

## Step 1: Dataprep

In [10]:
%%writefile dataprep.py
from azureml.core import Run

import pandas as pd 
import numpy as np 
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
import argparse

RANDOM_SEED=42

def prepare_age(df):
    # Fill in missing Age values from distribution of present Age values 
    mean = df["Age"].mean()
    std = df["Age"].std()
    is_null = df["Age"].isnull().sum()
    # compute enough (== is_null().sum()) random numbers between the mean, std
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = df["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    df["Age"] = age_slice
    df["Age"] = df["Age"].astype(int)
    
    # Quantize age into 5 classes
    df['Age_Group'] = pd.qcut(df['Age'],5, labels=False)
    df.drop(['Age'], axis=1, inplace=True)
    return df

def prepare_fare(df):
    df['Fare'].fillna(0, inplace=True)
    df['Fare_Group'] = pd.qcut(df['Fare'],5,labels=False)
    df.drop(['Fare'], axis=1, inplace=True)
    return df 

def prepare_genders(df):
    genders = {"male": 0, "female": 1, "unknown": 2}
    df['Sex'] = df['Sex'].map(genders)
    df['Sex'].fillna(2, inplace=True)
    df['Sex'] = df['Sex'].astype(int)
    return df

def prepare_embarked(df):
    df['Embarked'].replace('', 'U', inplace=True)
    df['Embarked'].fillna('U', inplace=True)
    ports = {"S": 0, "C": 1, "Q": 2, "U": 3}
    df['Embarked'] = df['Embarked'].map(ports)
    return df
    
parser = argparse.ArgumentParser()
parser.add_argument('--output_path', dest='output_path', required=True)
args = parser.parse_args()
    
titanic_ds = Run.get_context().input_datasets['titanic_ds']
df = titanic_ds.to_pandas_dataframe().drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
df = prepare_embarked(prepare_genders(prepare_fare(prepare_age(df))))

os.makedirs(os.path.dirname(args.output_path), exist_ok=True)
pq.write_table(pa.Table.from_pandas(df), args.output_path)

print(f"Wrote test to {args.output_path} and train to {args.output_path}")

Overwriting dataprep.py


In [11]:
prepped_data_path = PipelineData("titanic_train", datastore).as_dataset()

In [12]:
dataprep_step = PythonScriptStep(
    name="dataprep", 
    script_name="dataprep.py", 
    compute_target=compute, 
    runconfig=aml_run_config,
    arguments=["--output_path", prepped_data_path],
    inputs=[titanic_ds.as_named_input("titanic_ds")],
    outputs=[prepped_data_path],
    allow_reuse=True
)

### Step 2: Train with AutoMLStep

In [13]:
prepped_data_potds = prepped_data_path.parse_parquet_files(file_extension=None)

X = prepped_data_potds.drop_columns('Survived')
y = prepped_data_potds.keep_columns('Survived')


# Change iterations to a reasonable number (50) to get better accuracy
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 50,
    "experiment_timeout_hours" : 1,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations" : 2
}

automl_config = AutoMLConfig(task = 'classification',
                             path = '.',
                             debug_log = 'automated_ml_errors.log',
                             compute_target = compute,
                             run_configuration = aml_run_config,
                             featurization = 'auto',
                             X = X,
                             y = y,
                             **automl_settings)
                             
print("AutoML config created.")

WARNING - The AutoMLConfig inputs you have specified will soon be deprecated. Please use the AutoMLConfig shown in our documentation: https://aka.ms/AutoMLConfig


AutoML config created.


In [14]:
dstor = Datastore.get_default(ws)

metrics_data = PipelineData(name='metrics_data',
                           datastore=dstor,
                           pipeline_output_name='metrics_output',
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='best_model_data',
                           datastore=dstor,
                           pipeline_output_name='model_output',
                           training_output=TrainingOutput(type='Model'))


train_step = AutoMLStep(name='AutoML_Classification',
                                 automl_config=automl_config,
                                 passthru_automl_config=False,
                                 outputs=[metrics_data,model_data],
                                 allow_reuse=True)
print("train_step created.")

train_step created.


## Step 3: Register the model

In [15]:
%%writefile register_model.py
from azureml.core.model import Model, Dataset
from azureml.core.run import Run, _OfflineRun
from azureml.core import Workspace
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--model_name", required=True)
parser.add_argument("--model_path", required=True)
args = parser.parse_args()

print(f"model_name : {args.model_name}")
print(f"model_path: {args.model_path}")

run = Run.get_context()
ws = Workspace.from_config() if type(run) == _OfflineRun else run.experiment.workspace

model = Model.register(workspace=ws,
                       model_path=args.model_path,
                       model_name=args.model_name)

print("Registered version {0} of model {1}".format(model.version, model.name))


Overwriting register_model.py


In [16]:
# The model name with which to register the trained model in the workspace.
model_name = PipelineParameter("model_name", default_value="TitanicSurvival")

register_step = PythonScriptStep(script_name="register_model.py",
                                       name="register_model",
                                       allow_reuse=False,
                                       arguments=["--model_name", model_name, "--model_path", model_data],
                                       inputs=[model_data],
                                       compute_target=compute,
                                       runconfig=aml_run_config)

## Submit it

In [17]:
if not 'titanic_automl' in ws.experiments.keys() :
    Experiment(ws, 'titanic_automl')
experiment = ws.experiments['titanic_automl']

In [18]:
pipeline = Pipeline(ws, [dataprep_step, train_step, register_step])

In [19]:
run = experiment.submit(pipeline, show_output=True)

Created step dataprep [82923531][9feebadb-89b4-4e8f-ac9f-343109ef3210], (This step will run and generate new outputs)
Created step AutoML_Classification [7b2899fe][1664857a-ed0a-4049-a3f7-d721da2b5671], (This step will run and generate new outputs)
Created step register_model [46f6b217][9b02967c-08ec-4d84-b2d9-64e4a0de0435], (This step will run and generate new outputs)
Submitted PipelineRun 9375b0cb-b170-43a8-b93e-5e071aebd0c1
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/titanic_automl/runs/9375b0cb-b170-43a8-b93e-5e071aebd0c1?wsid=/subscriptions/65a1016d-0f67-45d2-b838-b8f373d6d52e/resourcegroups/laobri-ml/workspaces/pipelines


In [20]:
#run.wait_for_completion()

In [21]:
# automl_run = next(r for r in run.get_children() if r.name == 'AutoML_Classification')
# outputs = automl_run.get_outputs()
# metrics = outputs['default_metrics_AutoML_Classification']
# model = outputs['default_model_AutoML_Classification']

# metrics.get_port_data_reference().download('.')
# model.get_port_data_reference().download('.')

In [22]:
#metrics

In [23]:
#type(model)

In [24]:
#model